In [124]:
import numpy as np
import math as math
import sys
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline
import pandas as pd
np.set_printoptions(threshold=sys.maxsize)

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [125]:
df1 = pd.read_csv('input_data.csv')
inputs = df1.to_numpy()

df2 = pd.read_csv('output_data.csv')
outputs = df2.to_numpy()

print ('The shape of X is: ' + str(inputs.shape))
print ('The shape of y is: ' + str(outputs.shape))

df3 = pd.read_csv('input_validation_data.csv')
testIn = df3.to_numpy()
df4 = pd.read_csv('output_validation_data.csv')
testOut = df4.to_numpy()

The shape of X is: (16626, 367)
The shape of y is: (16626, 4)


In [127]:
model = Sequential(
    [               
        tf.keras.Input(shape=(367,)),    #specify input size
        ### START CODE HERE ### 
        Dense(150, activation='sigmoid'),
        Dense(125, activation='sigmoid'),
        Dense(100, activation='sigmoid'),
        Dense(75, activation='sigmoid'),
        Dense(50, activation='sigmoid'),
        Dense(25, activation='sigmoid'),
        Dense(4, activation='sigmoid')
        
        
        ### END CODE HERE ### 
    ], name = "my_model" 
)

In [128]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 150)               55200     
                                                                 
 dense_43 (Dense)            (None, 125)               18875     
                                                                 
 dense_44 (Dense)            (None, 100)               12600     
                                                                 
 dense_45 (Dense)            (None, 75)                7575      
                                                                 
 dense_46 (Dense)            (None, 50)                3800      
                                                                 
 dense_47 (Dense)            (None, 25)                1275      
                                                                 
 dense_48 (Dense)            (None, 4)                 104

In [129]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.legacy.Adam(0.001),
)
model.fit(
    inputs,outputs,
    epochs=250
)

Epoch 1/250
520/520 [==============================] - 1s 694us/step - loss: 0.5028
Epoch 2/250
520/520 [==============================] - 1s 2ms/step - loss: 0.4889
Epoch 3/250
520/520 [==============================] - 0s 785us/step - loss: 0.4222
Epoch 4/250
520/520 [==============================] - 0s 719us/step - loss: 0.4141
Epoch 5/250
520/520 [==============================] - 0s 714us/step - loss: 0.4112
Epoch 6/250
520/520 [==============================] - 0s 796us/step - loss: 0.4082
Epoch 7/250
520/520 [==============================] - 0s 720us/step - loss: 0.4048
Epoch 8/250
520/520 [==============================] - 0s 712us/step - loss: 0.4010
Epoch 9/250
520/520 [==============================] - 0s 719us/step - loss: 0.3978
Epoch 10/250
520/520 [==============================] - 0s 723us/step - loss: 0.3939
Epoch 11/250
520/520 [==============================] - 0s 712us/step - loss: 0.3855
Epoch 12/250
520/520 [==============================] - 0s 714us/step - loss

520/520 [==============================] - 0s 711us/step - loss: 0.2594
Epoch 98/250
520/520 [==============================] - 0s 719us/step - loss: 0.2550
Epoch 99/250
520/520 [==============================] - 0s 714us/step - loss: 0.2533
Epoch 100/250
520/520 [==============================] - 0s 716us/step - loss: 0.2545
Epoch 101/250
520/520 [==============================] - 0s 718us/step - loss: 0.2504
Epoch 102/250
520/520 [==============================] - 0s 715us/step - loss: 0.2516
Epoch 103/250
520/520 [==============================] - 0s 703us/step - loss: 0.2491
Epoch 104/250
520/520 [==============================] - 0s 711us/step - loss: 0.2496
Epoch 105/250
520/520 [==============================] - 0s 700us/step - loss: 0.2486
Epoch 106/250
520/520 [==============================] - 0s 718us/step - loss: 0.2451
Epoch 107/250
520/520 [==============================] - 0s 719us/step - loss: 0.2440
Epoch 108/250
520/520 [==============================] - 0s 715us/step

520/520 [==============================] - 0s 708us/step - loss: 0.1892
Epoch 193/250
520/520 [==============================] - 0s 708us/step - loss: 0.1855
Epoch 194/250
520/520 [==============================] - 0s 708us/step - loss: 0.1866
Epoch 195/250
520/520 [==============================] - 0s 728us/step - loss: 0.1850
Epoch 196/250
520/520 [==============================] - 0s 713us/step - loss: 0.1848
Epoch 197/250
520/520 [==============================] - 0s 707us/step - loss: 0.1856
Epoch 198/250
520/520 [==============================] - 0s 709us/step - loss: 0.1870
Epoch 199/250
520/520 [==============================] - 0s 708us/step - loss: 0.1836
Epoch 200/250
520/520 [==============================] - 0s 714us/step - loss: 0.1814
Epoch 201/250
520/520 [==============================] - 0s 725us/step - loss: 0.1822
Epoch 202/250
520/520 [==============================] - 0s 710us/step - loss: 0.1849
Epoch 203/250
520/520 [==============================] - 0s 712us/st

In [158]:
print ('The shape of X is: ' + str(testIn.shape))
prediction = model.predict(testIn[143].reshape(1,367))
print(f" predicting: {prediction}")

The shape of X is: (149, 367)
1/1 [==============================] - 0s 13ms/step
 predicting: [[1.7867002e-04 1.8936875e-01 6.6548833e-03 7.3547721e-01]]


In [154]:
def encode_input(p, pl, bet, h1, hs1, h2, hs2, c1, cs1, c2, cs2, c3, cs3, c4, cs4, c5, cs5):
    inp = [0.0] * 367
    inp[0] = p
    inp[1] = pl
    inp[2] = math.log(bet, 5)
    inp[2 + h1 + 13 * (hs1-1)] = 1
    inp[2 + 52 + h2 + 13 * (hs2-1)] = 1
    if c1 != 0:
        inp[2 + 52 * 2 + c1 + 13 * (cs1-1)] = 1
    if c2 != 0:
        inp[2 + 52 * 3 + c2 + 13 * (cs2-1)] = 1
    if c3 != 0:
        inp[2 + 52 * 4 + c3 + 13 * (cs3-1)] = 1
    if c4 != 0:
        inp[2 + 52 * 5 + c4 + 13 * (cs4-1)] = 1
    if c5 != 0:
        inp[2 + 52 * 6 + c5 + 13 * (cs5-1)] = 1
    return inp

In [159]:
x = np.array(encode_input(3, 2, 100, 13, 2, 1, 4, 6, 2, 12, 3, 2, 1, 1, 3, 0, 0))


In [160]:
prediction = model.predict(x.reshape(1,367))
print(f" predicting: {prediction}")

1/1 [==============================] - 0s 50ms/step
 predicting: [[1.7867002e-04 1.8936875e-01 6.6548833e-03 7.3547721e-01]]
